# Calculs sur un Modèle Numérique de Terrain (MNT) 
# Projet première partie

## M1 informatique, Université d'Orléans 2021/2022

**Ce projet est à faire en binôme. Vous aurez plusieurs rendus à déposer sur Celene mais avec un seul rendu par binôme.**

Un MNT est une matrice représentant une parcelle de terrain dont chaque élément représente la hauteur en un point dit géo-référencé du terrain. Nous allons utilisé dans ce projet des MNT stockés dans des fichiers .txt avec une entête indiquant les différents paramètres du MNT puis les données de hauteur par ligne de la parcelle

Les deux premières valeurs indiquent la taille de la parcelle en nombre de lignes et de colonnes. Les deux lignes suivantes indiquent les coordonnées du coin supérieur gauche et la cinquième ligne la taille du point. Ces informations ne seront pas utilisées. Enfin la dernière ligne de l’entête donne la valeur appelée **NO_DATA** associée à l’absence d’information sur un point du MNT. Les lignes suivantes contiennent les valeurs des hauteurs. 

Les bassins versants sont des surfaces closes délimitées par des lignes de partages des eaux et qui sont
entièrement caractérisées par leurs exutoires c’est-à-dire la zone de convergence des eaux. C'est un calcul classique sur les MNT et il se décompose en plusieurs étapes dont le calcul des directions et le calcul des flots d'accumulation. 

### 1. les directions 

Il s'agit pour chaque point du MNT de coder la direction du flux. Autrement dit on recherche parmi les 8 voisins le point le plus bas. Le résultat est une direction Nord, Nord Est, Est, etc codée respectivement par un entier de 1 à 8 comme illustré ci-dessous. Si le point correspond à une cuvette c’est-à-dire s’il est plus bas que ses voisins alors on codera la direction par un 0.


| Direction     |     Code        | 
| ------------- |: -------------: | 
| Nord          |        1        |
| Nord-Est      |        2        | 
| Est           |        3        |  
| Sud-Est       |        4        |  
| Sud           |        5        |  
| Sud-Ouest     |        6        |  
| Ouest         |        7        |  
| Nord-Ouest    |        8        |  
| Cuvette       |        0        |  

et par rapport à une cellule 
$$\begin{array}{|c|c|c|} 
\hline
8 &1 &2\\
\hline
7 &0 &3\\
\hline
6 &5 &4\\
\hline
\end{array}$$

En considérant un MNT de taille $n \times m$ et si on note une cellule du MNT par ses indices $(i,j)$ alors $H(i,j)$ désigne la hauteur du point $(i,j)$ et le calcul des directions est une matrice $D$ telle que $D(i,j)$ est la direction du point $(i,j)$ dans le MNT.

## 2. Les flots d'accumulation

La détermination des flots d’accumulation consiste à calculer pour chaque cellule d’un MNT le nombre de cellules qu’elle draine (autrement dit qui se déversent dans cette cellule). Ce calcul nécessite de connaître le résultat du calcul des directions. On peut alors appliquer l'algorithme suivant avec $F$ la matrice des flots d'accumulation telle que $F(i,j)$ est la somme des points qui se déversent dans $(i,j)$.

1. Initialiser $F$ : 
    * $F(i,j)=1$ pour tous les points qui n'ont pas de voisins plus hauts qu'eux sinon $F(i,j)=0$. On dit alors qu'un point est marqué si $F(i,j) \neq 0$
    
2. Tant qu'il existe un point $(i,j)$ non marqué
    * Pour chaque point $(i,j)$ non marqué, si tous les voisins qui se déversent sur lui sont marqués, alors  $F(i,j)$ est la somme des flots de ces voisins plus lui même.
    
Voici un exemple avec le codage des directions par des flèches afin de faciliter la compréhension du calcul des flots d'accumulation

<img src="exemple.png" alt="drawing" width="600"/>
    
## 3. La parallélisation

On souhaite paralléliser ces deux calculs en utilisant une architecture hybride. Autrement dit le MNT pouvant représenter un gros volume de données on souhaiterait distribuer le MNT initial sur les différents noeuds d'une machine parallèle mais également paralléliser les calculs effectués sur chaque noeud en utilisant les coeurs disponibles. 

Proposez une parallélisation complète de ces deux calculs sachant qu'on souhaite les faire l'un à la suite de l'autre et qu'on souhaite distribuer le MNT initial par bande sur les différents processeurs de la machine parallèle. La distribution pour mieux répartir les *NO_DATA* devra suivre **une distribution en round-robin**.

Avant de débuter l'implémentation, vous allez rédiger un petit rapport décrivant votre parallélisation et répondant en particulier aux questions suivantes. Il s'agit de prendre le temps de lire le sujet et de poser les bases avant de coder.

1. Si le processus *root* seul peut lire le fichier *.txt* contenant le MNT quelles communications seront nécessaires sachant que seule la lecture du fichier donne les informations sur la taille du MNT ? Vous pourrez par contre faire l'hypothèse qu'on travaillera sur des terrains dont le nombre de lignes est une puissance de 2. 
2. Quelles hypothèses ferez vous pour permettre une gestion simple de la communication en round-robin (taille d'une bande en nombre de lignes, nombre de bandes par processus, nombre de processus, ...) ?
3. Quelle sera la taille du tableau contenant les morceaux de MNT attribués à chaque processus ?
4. Comment allez vous gérer les bords horizontaux ? Est-ce que certains processus devront réaliser une communication particulière ?
5. Comment allez vous calculer la direction d'un point du MNT lorsque c'est un point avec huit voisins ?
6. Comment allez vous calculer la direction d'un point du MNT lorsque ce point est sur un bord vertical avec 5 voisins ?

7. Pour le calcul des flots d'accumulation, chaque processus aura à sa disposition ses morceaux de MNT et les directions qu'il a calculées. Sera-t-il nécessaire de faire de nouvelles communications avant de commencer le calcul des flots ?
8. Quelles seront les échanges à effectuer au cours des itérations du calcul des flots ?
9. Comment l'ensemble des processus sauront que le calcul des flots est terminé et que tous les points sont marqués ?
10. A partir de la direction des huit voisins comment allez vous déterminer si un point a des points plus hauts qui se déversent sur lui.
11. Quelles seront les parties que vous pourrez réaliser en OpenMP ? Quel gain pensez vous obtenir en utilisant une implémentation hybride ?



**Rapport à rendre à 10h30 sur Celene (dépôt RapportRendu1) avec la constitution de votre binôme**
